# Logging Example

### get AWS key and  make session


In [22]:
import json, boto3

with open("./.KEYS/WATCHER_ACCESS_KEY.json", "r") as f:
    key = json.load(f)
with open("./.KEYS/DATA_SRC_INFO.json", "r") as f:
    bucket_info = json.load(f)
session = boto3.Session(
    aws_access_key_id=key['aws_access_key_id'],
    aws_secret_access_key=key['aws_secret_key'],
    region_name=key['region']
)
# 
client = session.client('logs')

### put log event with boto3 client

In [39]:
from datetime import datetime, timedelta
import time
query = "fields @timestamp, @message | parse @message \"username: * ClinicID: * nodename: *\" as username, ClinicID, nodename | filter ClinicID = 7667 and username='simran+test@example.com'"

# make sure to exist 
# log group /aws/{cartegory}/{site_name}
log_group = '/aws/lambda/crawler-jobkorea'
log_stream = 'test'

# if log_group or log_stream doesn't exist, create them
try:
    client.create_log_group(logGroupName=log_group)
except client.exceptions.ResourceAlreadyExistsException:
    pass

try:
    client.create_log_stream(logGroupName=log_group, logStreamName=log_stream)
except client.exceptions.ResourceAlreadyExistsException:
    pass

# set your message
msg = "Hello Test"

# push log event 
response = client.put_log_events(
    logGroupName=log_group,
    logStreamName=log_stream,
    logEvents=[
        {
            'timestamp': int(round(time.time() * 1000)),
            'message': msg
        },
    ],
)
    

You must check response about requset. 
- if it has 'rejectedLogEvents', log event doesn't be allowed
- make sure that your log event has right timestamp: recomanded method is `int(round(time.time() * 1000))`

In [40]:
response 

{'nextSequenceToken': '49655348470361518325449427548644439034907206660153933842',
 'ResponseMetadata': {'RequestId': 'eca1f845-9e57-4fa6-885b-d1ddd00e963d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'eca1f845-9e57-4fa6-885b-d1ddd00e963d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '80',
   'date': 'Fri, 30 Aug 2024 06:05:28 GMT'},
  'RetryAttempts': 0}}

In [ ]:
import logging
# import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError

# CloudWatch Logs 클라이언트 생성
# client = boto3.client('logs', region_name='ap-northeast-2')  # 서울 리전
import json, boto3

with open("./.KEYS/WATCHER_ACCESS_KEY.json", "r") as f:
    key = json.load(f)
with open("./.KEYS/DATA_SRC_INFO.json", "r") as f:
    bucket_info = json.load(f)
session = boto3.Session(
    aws_access_key_id=key['aws_access_key_id'],
    aws_secret_access_key=key['aws_secret_key'],
    region_name=key['region']
)
# 
client = session.client('logs')

# 로그 그룹과 스트림 이름 설정
# log_group_name = 'your-log-group'
# log_stream_name = 'your-log-stream'
log_group_name = '/aws/lambda/crawler-jobkorea'
log_stream_name = 'test'

# 로그 그룹과 스트림이 존재하지 않으면 생성
try:
    client.create_log_group(logGroupName=log_group_name)
except client.exceptions.ResourceAlreadyExistsException:
    pass

try:
    client.create_log_stream(logGroupName=log_group_name, logStreamName=log_stream_name)
except client.exceptions.ResourceAlreadyExistsException:
    pass

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class CloudWatchHandler(logging.Handler):
    def emit(self, record):
        log_entry = self.format(record)
        try:
            response = client.put_log_events(
                logGroupName=log_group_name,
                logStreamName=log_stream_name,
                logEvents=[
                    {
                        'timestamp': int(record.created * 1000),
                        'message': log_entry
                    },
                ],
            )
        except (NoCredentialsError, PartialCredentialsError) as e:
            print(f"Credentials error: {e}")
        except Exception as e:
            print(f"Error sending log to CloudWatch: {e}")

# CloudWatchHandler 추가
cloudwatch_handler = CloudWatchHandler()
cloudwatch_handler.setLevel(logging.INFO)
logger.addHandler(cloudwatch_handler)

# 예제 로그
logger.info("This is an info message from usernamehey")
logger.error("This is an error message from usernamehey")
